In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [5]:
# Filter for Total Votes > 20
vinedf2= vine_df.filter(vine_df.total_votes >= 20)
vinedf2.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
# Filter for helpful/ total = 50%
vinedf3 = vinedf2.filter((vinedf2.helpful_votes / vinedf2.total_votes) >= 0.5)
vinedf3.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [16]:
# Create a DF of paid Vine reviews
vine_paid_df = vinedf3.filter(vinedf3.vine == 'Y')
vine_paid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RPJ7C4HP3BHXN|          5|           25|         29|   Y|                N|
|R3SB08XK0M7993|          4|           44|         49|   Y|                N|
|R26X1DACMCN3YF|          3|           16|         21|   Y|                N|
|R17HKKDE9ZRVOO|          4|           31|         35|   Y|                N|
|R1R7GNSCMO8U7I|          4|           24|         28|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [9]:
# Create a DF of unpaid Vine reviews
vine_unpaid_df = vinedf3.filter(vinedf3.vine == 'N')
vine_unpaid_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [10]:
# Total number of reviews(overall)
total_reviews = vinedf3.count()
total_reviews

6711

In [11]:
# Total number of reviews(overall)
total_paid = vine_paid_df.count()
total_paid

21

In [12]:
# Total number of reviews(overall)
total_unpaid = vine_unpaid_df.count()
total_unpaid

6690

In [13]:
# Number of 5-star reviews (overall)
total_five_star = vinedf3.filter(vinedf3.star_rating == 5).count()
total_five_star

3458

In [14]:
# Number of 5-star reviews (overall)
paid_five_star = vine_paid_df.filter(vine_paid_df.star_rating == 5).count()
paid_five_star

10

In [17]:
# Number of 5-star reviews (overall)
unpaid_five_star = vine_unpaid_df.filter(vine_unpaid_df.star_rating == 5).count()
unpaid_five_star

3448

In [28]:
# % of overall 5-star reviews
total_five_star_percent = (total_five_star / total_reviews) * 100
total_five_star_percent

51.527343167933246

In [ ]:
# % of paid 5-star reviews
paid_five_star_percent = (paid_five_star / total_paid) * 100
paid_five_star_percent

47.61904761904761

In [19]:
# % of unpaid 5-star reviews
unpaid_five_star_percent = (unpaid_five_star / total_unpaid) * 100
unpaid_five_star_percent

51.53961136023917

In [29]:
print("Total reviews: " + str(total_reviews) + ". 5-star reviews: " + str(total_five_star) + ". Percent of 5-star reviews: " + str(total_five_star_percent))

Total reviews: 6711. 5-star reviews: 3458. Percent of 5-star reviews: 51.527343167933246


In [ ]:
print("Paid Vine reviews: " + str(total_paid) + ". Paid 5-star reviews: " + str(paid_five_star) + ". Percent of Paid 5-star reviews: " + str(paid_five_star_percent))

Paid Vine reviews: 21. Paid 5-star reviews: 10. Percent of Paid 5-star reviews: 47.61904761904761


In [27]:
print("Unpaid Vine reviews: " + str(total_unpaid) + ". Unpaid 5-star reviews: " + str(unpaid_five_star) + ". Percent of unpaid 5-star reviews: " + str(unpaid_five_star_percent))

Unpaid Vine reviews: 6690. Unpaid 5-star reviews: 3448. Percent of unpaid 5-star reviews: 51.53961136023917
